In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.0


In [2]:
classification_task = mz.tasks.Classification(num_classes=2)
classification_task.metrics = ['acc']
print("`classification_task` initialized with metrics", classification_task.metrics)

`classification_task` initialized with metrics [accuracy]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.quora_qp.load_data('train', task=classification_task)
dev_pack_raw = mz.datasets.quora_qp.load_data('dev', task=classification_task)
test_pack_raw = mz.datasets.quora_qp.load_data('test', task=classification_task)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...


b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'


data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.ESIM.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 266106/266106 [00:38<00:00, 6984.81it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 273121/273121 [00:36<00:00, 7382.63it/s]
Building Vocabulary from a datapack.: 100%|██████████| 6022311/6022311 [00:01<00:00, 3638545.75it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 266106/266106 [00:34<00:00, 7678.60it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 273121/273121 [00:36<00:00, 7461.54it/s]
Processing length_right with len: 100%|██████████| 273121/273121 [00:00<00:00, 960189.79it/s] 
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 37693/37693 [00:04<00:00, 8180.50it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7faef87e7da0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7faed60050f0>,
 'vocab_size': 78764,
 'embedding_input_dim': 78764}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='point'
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed,
    mode='point'
)

In [9]:
padding_callback = mz.models.ESIM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=40,
    stage='train',
    sort=False,
    callback=padding_callback
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    batch_size=40,
    stage='dev',
    sort=False,
    callback=padding_callback
)

In [10]:
model = mz.models.ESIM()

model.params['task'] = classification_task
model.params['embedding'] = embedding_matrix
model.params['mask_value'] = 0
model.params['dropout'] = 0.2
model.params['hidden_size'] = 200
model.params['lstm_layer'] = 1

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ESIM(
  (embedding): Embedding(78764, 100, padding_idx=0)
  (rnn_dropout): RNNDropout(p=0.2, inplace=False)
  (input_encoding): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(100, 100, bidirectional=True)
    )
  )
  (attention): BidirectionalAttention()
  (projection): Sequential(
    (0): Linear(in_features=800, out_features=200, bias=True)
    (1): ReLU()
  )
  (composition): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(200, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=800, out_features=200, bias=True)
    (2): Tanh()
    (3): Dropout(p=0.2, inplace=False)
  )
  (out): Linear(in_features=200, out_features=2, bias=True)
)
Trainable params:  8600402


In [11]:
optimizer = torch.optim.Adam(model.parameters(),lr=1e-5)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=5
)

In [12]:
trainer.run()

[Iter-9079 Loss-0.609]:
  Validation: accuracy: 0.7135



[Iter-18158 Loss-0.555]:
  Validation: accuracy: 0.7288



[Iter-27237 Loss-0.536]:
  Validation: accuracy: 0.7375



[Iter-36316 Loss-0.524]:
  Validation: accuracy: 0.744



[Iter-45395 Loss-0.515]:
  Validation: accuracy: 0.7494

Cost time: 27346.551464796066s
